In [63]:
# TO DO:
# - lav regex/funktion til at dele korrekt ved sætninger
# - kør vader på hver sætning og returner gennemsnitlig compound

import numpy as np

In [132]:
import json
import os
import time
import string
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import re
import pickle

# Alpha-num, lowercase
def easy_clean(comment):
    comment = re.sub('[^A-Za-z ]+', '', comment['body']).lower().split(' ')
    #comment = [re.sub('[^A-Za-z]+', '', word) for word in comment]
    return comment

# nltk tokenize, stopwords
def nltk_clean(comment):
    #comment = comment
    #comment = nltk.sent_tokenize(comment)
    print(comment)
    comment = nltk.tokenize.word_tokenize(comment)
    comment = [word for word in comment if word.isalpha()]
    stop_words = nltk.corpus.stopwords.words('english')
    comment = [word for word in comment if word not in stop_words]
    return comment

# Remove quotes, clean text using translation table (punctuations)
def nltk2_clean(comment): #tokenize, removes case, remove special characters and numbers
    comment = [e for e in comment.splitlines() if e[:2] != '&g']
    comment = ' '.join(comment)
    comment = comment.split(' ')
    comment = [w.lower() for w in comment]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in comment]
    words = [word for word in stripped if word.isalpha()]
    return ' '.join(words)

# Tokenize at sentence level, applies nltk2_clean on each sentence
def clean_sent(comment): #tokenize, removes case, remove special characters and numbers
    comment = comment['body']
    comments = nltk.sent_tokenize(comment)
    comments = [nltk2_clean(sent) for sent in comments]
    return comments


# SÆTNINGSDELING REGEX: (?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s

#    sentence_list = tokenize.sent_tokenize(paragraph)
#    paragraphSentiments = 0.0
#    for sentence in sentence_list:
#        vs = analyzer.polarity_scores(sentence)
#        print("{:-<69} {}".format(sentence, str(vs["compound"])))
#        paragraphSentiments += vs["compound"]
#    print("AVERAGE SENTIMENT FOR PARAGRAPH: \t" + str(round(paragraphSentiments / len(sentence_list), 4)))





In [130]:
# Remove duplicates across dataSet

clSet = [] 
crSet = []
trSet = []
saSet = []


print('starter')
dataSet = []
start = time.time()
n = 0
for file in os.listdir('data/'):
    with open('data/'+file) as f:
        data = json.load(f)
    
    if file[:2] == 'cl':
        candidates =  ['bernie' ,'sanders' ,'donald' ,'trump' ,'ted' ,'cruz']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        clData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        clSet = clSet + clData
        
    elif file[:2] == 'cr':
        candidates =  ['bernie' ,'sanders' ,'donald' ,'trump', 'hillary', 'clinton']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        crData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        crSet = crSet + crData

    elif file[:2] == 'tr':
        candidates =  ['bernie' ,'sanders' ,'ted' ,'cruz', 'hillary', 'clinton']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        trData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        trSet = trSet + trData
        
    elif file[:2] == 'sa':
        candidates =  ['hillary', 'clinton' ,'donald' ,'trump' ,'ted' ,'cruz']
        #tempData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        saData = [e for e in data if not any(item in candidates for item in easy_clean(e))]
        saSet = saSet+saData
    print(file + '\ndone\n')
    
end = time.time()
print('...Done in '+str(end - start))



starter
clinton_politics_2016-01-01_2016-02-01.json
done

clinton_politics_2016-02-01_2016-03-01.json
done

clinton_politics_2016-03-01_2016-04-01.json
done

clinton_politics_2016-04-01_2016-05-01.json
done

clinton_politics_2016-05-01_2016-06-01.json
done

clinton_politics_2016-06-01_2016-07-01.json
done

clinton_politics_2016-07-01_2016-08-01.json
done

clinton_politics_2016-08-01_2016-09-01.json
done

clinton_politics_2016-09-01_2016-10-01.json
done

clinton_politics_2016-10-01_2016-11-01.json
done

clinton_politics_2016-11-01_2016-12-01.json
done

clinton_politics_2016-12-01_2016-12-31.json
done

cruz_politics_2016-01-01_2016-02-01.json
done

cruz_politics_2016-02-01_2016-03-01.json
done

cruz_politics_2016-03-01_2016-04-01.json
done

cruz_politics_2016-04-01_2016-05-01.json
done

cruz_politics_2016-05-01_2016-06-01.json
done

cruz_politics_2016-06-01_2016-07-01.json
done

cruz_politics_2016-07-01_2016-08-01.json
done

cruz_politics_2016-08-01_2016-09-01.json
done

cruz_politics_20

In [134]:
# Save data

dataSet = [clSet, crSet, trSet, saSet]
with open("nodupData", "wb") as fp:
    pickle.dump(dataSet, fp)

In [76]:
# Import vader sentiment analyser
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sentences = ['that food was awful','what a beautiful dress','lock her up!','she is a shill','thomas is not very bright']
analyzer = SIA()
sent = [[' '.join(sentence), analyzer.polarity_scores(sentence[0])] for sentence in cleanData]

# Calculate polarity score of each sentence
sentScores = [[[sentence, analyzer.polarity_scores(sentence)['compound']]
             for sentence in comment if len(sentence)>1] for comment in cleanData]

# Calculate average polarity score for each sentence in comment + comment in string format
commentScores = []
for comment in sentScores:
    sentence = '. '.join([sentence[0] for sentence in comment])
    avg = np.mean([sentence[1] for sentence in comment])
    commentScores.append([sentence, avg])

print(len(commentScores))    
print(commentScores[0])
print('\n')
print(sentScores[0])




9600
['you are twisting them by equating regular staffers versus astroturfing. so are you saying its ethical for a presidential candidate to try and manipulate public perception through anonymous boards. it doesnt matter if astrotufing gains hillary vote or votes. its not ethical to employ individuals to act as regular people with no direct affiliation', 0.058724999999999999]


[['you are twisting them by equating regular staffers versus astroturfing', 0.0], ['so are you saying its ethical for a presidential candidate to try and manipulate public perception through anonymous boards', 0.5106], ['it doesnt matter if astrotufing gains hillary vote or votes', 0.3239], ['its not ethical to employ individuals to act as regular people with no direct affiliation', -0.5996]]


In [123]:
# Sort by compound
from operator import itemgetter

commentScores.sort(key=lambda e: e[1],reverse=True)

i = -9
print(commentScores[i])

['if trump wins hell probably end up trying to abuse his position and get snl taken off the air considering his repeatedly expressed desire to expand libel laws in order to attack anyone who criticizes him', -0.76500000000000001]
